In [1]:
import pandas as pd
import sqlite3

In [2]:
#établissement de la connexion à la base de données et création du curseur
conn = sqlite3.connect("haikintana.db")
c= conn.cursor()
#conn.row_factory=sqlite3.Row

In [4]:
#création des tables
c.execute("CREATE TABLE IF NOT EXISTS membre (ID text PRIMARY KEY, cotisation text, nom text, prénoms text, email text, bénévolat text)")
c.execute("CREATE TABLE IF NOT EXISTS activité(intitulé text PRIMARY KEY, date text, lieu text)")
c.execute("CREATE TABLE IF NOT EXISTS pôle(nom_pôle text PRIMARY KEY, nombre_activités integer)")
c.execute("CREATE TABLE IF NOT EXISTS membre_activité(ID_membre text, activité text , FOREIGN KEY (ID_membre) REFERENCES membre(ID), FOREIGN KEY (activité) REFERENCES activité(intitulé), PRIMARY KEY(ID_membre, activité))")
c.execute("CREATE TABLE IF NOT EXISTS membre_pôle(ID_membre text, pôle text, FOREIGN KEY (ID_membre) REFERENCES membre(ID), FOREIGN KEY (pôle) REFERENCES pôle(nom_pôle), PRIMARY KEY(ID_membre, pôle))")
c.execute("CREATE TABLE IF NOT EXISTS pôle_activité(pôle text, activité text, FOREIGN KEY (pôle) REFERENCES pôle(nom_pôle), FOREIGN KEY (activité) REFERENCES activité(intitulé), PRIMARY KEY(pôle, activité))")

In [5]:
#importation des données à partir du fichier excel et transfert vers la base
data=pd.read_excel('C:/Users/Harivola RABEKOTO/Liste HAIKINTANA 2023.xlsx')
data=data.drop(['Pôles','Commentaire'], axis=1) #on ne prend pas en compte les deux dernières colonnes du fichier excel
data=data.rename(columns={'Numéro':'ID','E-mail':'email', 'Prénom(s)':'prénoms'}) #on change le label 'numéro' en 'ID' et celui 'e-mail' en 'email' pour faire correspondre cette colonne à celle de la table
print(data)
data.to_sql('membres', conn, if_exists='replace', index=False) #on ne prend pas en compte les index du dataframe dans la table

         ID                  Cotisation                        Nom  \
0   HK-0000                         NaN                 RAZANAKOTO   
1   HK-0001                         NaN              RAJAONARIVELO   
2   HK-0002                         NaN                Ralambomamy   
3   HK-0003                         NaN             Rafieferantsoa   
4   HK-0004                         NaN                  Rabarison   
5   HK-0005                        payé             Andriamandroso   
6   HK-0006                         NaN                  Rakotoson   
7   HK-0007                   durant AG                    RABEONY   
8   HK-0008                   durant AG          Randriamanantena    
9   HK-0009                   durant AG           RANDRIAMANANTENA   
10  HK-0010               mvola(09mars)                 Rambeloson   
11  HK-0011                   durant AG            IALISOLONDRAIBE   
12  HK-0012                   durant AG         randrianandrasana    
13  HK-0013         

In [6]:
#ajout des colonnes 'statut' et 'assiduité'
c.execute("alter table membre add statut text")
c.execute("alter table membre add assiduité integer")

OperationalError: duplicate column name: statut

In [7]:
#remplissage de la table pôle
liste_pôles=[
    ('éducation',0),
    ('outreach',0),
    ('observation et instrumentation',0),
    ('développement et partenariat',0)
]
c.executemany("INSERT INTO pôle values(?,?)",liste_pôles)

IntegrityError: UNIQUE constraint failed: pôle.nom_pôle

In [8]:
#remplacement des valeurs nulles en un texte "aucun"
c.execute('UPDATE membre SET cotisation="non payé" WHERE  cotisation IS NULL')
c.execute('UPDATE membre SET email="inconnu" WHERE email IS NULL')
c.execute('UPDATE membre SET bénévolat="inconnu" WHERE bénévolat IS NULL')
c.execute('UPDATE membre SET statut="aucun"')  
c.execute('UPDATE membre SET assiduité=0 ')
conn.commit()

In [4]:
#affichage d'informations
def info_membre(id_recherche):
    c.execute("SELECT * FROM membre WHERE (ID)= (?)",(id_recherche,))
    informations_membre=c.fetchall()
    #using for loop to move out of the list 
    for i in informations_membre:
        print('Nom: '+ i[2] + '\n'+
              'Prénoms: '+ i[3] + '\n'+
             'Contact: '+ i[4] + '\n'+
             'Cotisation: '+ i[1] + '\n'+
             'Bénévolat: '+ i[5] + '\n'+
             'Statut: '+ i[6] + '\n' +
             'Nombre d\'activités assistées: '+ str(i[7]) + '\n')
info_membre('HK-0002')

Nom: Ralambomamy
Prénoms: Tyffani
Contact: inconnu
Cotisation: non payé
Bénévolat: inconnu
Statut: vice-présidente
Nombre d'activités assistées: 0



In [4]:
def liste_selon_assiduité():
    c.execute("SELECT ID, nom, prénoms, assiduité FROM membre ORDER BY assiduité DESC")
    liste_assiduite=c.fetchall()
    print('Identifiant \t Nom \t\t\t Prénoms \t Nombre d\'activités assistées')
    for i in liste_assiduite:
        print(i[0] + '\t\t' + i[1] + '\t\t' + i[2] + '\t\t' + str(i[3]) )
liste_selon_assiduité()

Identifiant 	 Nom 			 Prénoms 	 Nombre d'activités assistées
HK-0001		RAJAONARIVELO		Andoniaina		1
HK-0004		Rabarison		Toky 		1
HK-0007		RABEONY		Miora Heritiana		1
HK-0011		IALISOLONDRAIBE		Jean Miguel Haydon		1
HK-0013		RANDRIAMANARIVO		Mamitina Rolando		1
HK-0030		Randrianandraina		Andrea 		1
HK-0033		Rakotoarisoa		Zo anja avotriniaina		1
HK-0034		RABARISON		Dina Mandresy		1
HK-0035		RABEKOTO 		Harivola Ny Aina Céloréa 		1
HK-0038		RABARISOA		Anjia Randia		1
HK-0040		RALAIMANAMPISOA		Herinandrianina Tojomanana		1
HK-0041		RAZAFINDRAIBE		Olivier Ihary		1
HK-0043		RATSIZAFY 		Lazandrainy Nathanael 		1
HK-0048		Andriatiana		Mamy		1
HK-0049		NY ANDONAVALONA 		Hasina Gabrielle 		1
HK-0000		RAZANAKOTO		Mializo		0
HK-0002		Ralambomamy		Tyffani		0
HK-0003		Rafieferantsoa		Deralaza 		0
HK-0005		Andriamandroso		Mianoka		0
HK-0006		Rakotoson		Ny Tsiky		0
HK-0008		Randriamanantena 		Herman 		0
HK-0009		RANDRIAMANANTENA		Volasoa Elia		0
HK-0010		Rambeloson		Koloina Razafinjato		0
HK-0012		randri

In [3]:
#modification des données
def modification_membre(id_recherche,colomne,donnee):
    if colomne==1:
        c.execute("UPDATE membre SET (nom)=(?) WHERE (ID)=(?)",(donnee,id_recherche,))
    elif colomne==2:
        c.execute("UPDATE membre SET (prénoms)=(?) WHERE (ID)=(?)",(donnee,id_recherche,))
    elif colomne==3:
        c.execute("UPDATE membre SET (email)=(?) WHERE (ID)=(?)",(donnee,id_recherche,))
    elif colomne==4:
        c.execute("UPDATE membre SET (cotisation)=(?) WHERE (ID)=(?)",(donnee,id_recherche,))
    elif colomne==5:
        c.execute("UPDATE membre SET (bénévolat)=(?) WHERE (ID)=(?)",(donnee,id_recherche,))
    elif colomne==6:
        c.execute("UPDATE membre SET (statut)=(?) WHERE (ID)=(?)",(donnee,id_recherche,))
    elif colomne==7:
        c.execute("UPDATE membre SET (assiduité)=(?) WHERE (ID)=(?)",(donnee,id_recherche,))
    else:
        colomne=int(input('Entrez uniquement un chiffre de 1 à 7 \n'))
        modification_membre(id_recherche,colomne,donnee)
    conn.commit()
modification_membre('HK-0002',8, 'vice-présidente')

Entrez uniquement un chiffre de 1 à 7 
6


In [ ]:
#ajout de ligne dans la table membres
def ajout(liste_info):
    c.execute('INSERT INTO membre(ID, cotisation, nom, prénoms, email, bénévolat, statut) VALUES(?,?,?,?,?,?,?)',(liste_info))

In [9]:
#insertion d'informatioins d'une activité
def creation_activite(intitule, date, lieu):
    c.execute('INSERT INTO activité VALUES (?,?,?)',(intitule,date,lieu))
    conn.commit()
    
creation_activite('On the moon again 2023', '23/06/2023', 'La City Ivandry')

In [4]:
#relation entre une pôle et une activité et incréméntation du nombre d'activités de la pôle concernée
def pole_organisation(pole, intitule):
    if pole==1:
        nom_pole="outreach"
    elif pole==2:
        nom_pole="éducation"
    elif pole==3:
        nom_pole="observation et instrumentation"
    elif pole==4:
        nom_pole="développement et partenariat"
    c.execute('INSERT INTO pôle_activité VALUES (?,?)',(nom_pole,intitule))
    c.execute('SELECT nombre_activités FROM pôle WHERE nom_pôle=(?)',(nom_pole,))
    query_result=c.fetchall()
    for i in query_result:
        nombre=i[0]+1
    c.execute('UPDATE pôle SET nombre_activités=(?) WHERE nom_pôle=(?)',(nombre,nom_pole,))
    conn.commit()
pole_organisation('observation et instrumentation', 'On the moon again 2023')

UnboundLocalError: local variable 'nombre' referenced before assignment

In [ ]:
#relation entre un membre et une activité et incréméntation de l'assiduité du membre concerné
def presence(id_present,intitule):
    c.execute('INSERT INTO membre_activité VALUES (?,?)',(id_present,intitule))
    c.execute('SELECT assiduité FROM membre WHERE ID=(?)',(id_present, ))
    query_result=c.fetchall()
    for i in query_result:
        nombre=i[0]+1
    c.execute('UPDATE membre SET assiduité=(?) WHERE ID=(?)',(nombre,id_present,))
    conn.commit()

In [17]:
#affichage de la liste de présence à une activité donnée en paramètre et le nombre de membres l'ayant assisté
def liste_presence(intitule):
    c.execute('SELECT COUNT(*) FROM membre_activité WHERE activité=(?)',(intitule,))
    nombre=c.fetchall()
    for i in nombre:
        nombre=i[0]
    print('Nombre de membres présents:',nombre)
    c.execute('SELECT ID, nom, prénoms FROM membre WHERE ID IN (SELECT ID_membre FROM membre_activité WHERE activité=(?))',(intitule,))
    liste=c.fetchall()
    print ('Liste de présence à l\'activité:')
    print('Identifiant \t Nom \t\t\t Prénoms ')
    for i in liste:
        print(i[0] + '\t\t' + i[1] + '\t\t' + i[2] )
liste_presence('Formation Besely 1')

Nombre de membres présents: 15
Liste de présence à l'activité:
Identifiant 	 Nom 			 Prénoms 
HK-0001		RAJAONARIVELO		Andoniaina
HK-0004		Rabarison		Toky 
HK-0007		RABEONY		Miora Heritiana
HK-0011		IALISOLONDRAIBE		Jean Miguel Haydon
HK-0013		RANDRIAMANARIVO		Mamitina Rolando
HK-0030		Randrianandraina		Andrea 
HK-0033		Rakotoarisoa		Zo anja avotriniaina
HK-0034		RABARISON		Dina Mandresy
HK-0035		RABEKOTO 		Harivola Ny Aina Céloréa 
HK-0038		RABARISOA		Anjia Randia
HK-0040		RALAIMANAMPISOA		Herinandrianina Tojomanana
HK-0041		RAZAFINDRAIBE		Olivier Ihary
HK-0043		RATSIZAFY 		Lazandrainy Nathanael 
HK-0048		Andriatiana		Mamy
HK-0049		NY ANDONAVALONA 		Hasina Gabrielle 


In [18]:
#fermeture de la connexion
conn.close()